In [1]:
# Imports
import os
from dotenv import load_dotenv
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F

In [2]:
# Load environment variables
load_dotenv()

account=os.getenv('account')
user= os.getenv('user')
password=os.getenv('password')
warehouse=os.getenv('warehouse')

In [3]:
# Configuration for the Snowpark session
connection_parameters = {
    "account": os.getenv('account'),
    "user": os.getenv('user'),
    "password": os.getenv('password'),
}

In [22]:
# Initialize session
session = Session.builder.configs(connection_parameters).create()

# Snowpark DataFrame
healthcare_data_df = session.table("HEALTHCARE_DATASET.PUBLIC.HEALTHCARE_DATASET")

In [5]:
# View general structure of the dataset
healthcare_data_df.limit(5).show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"NAME"         |"AGE"  |"GENDER"  |"BLOODTYPE"  |"MEDICALCONDITION"  |"DATEOFADMISSION"  |"DOCTOR"          |"HOSPITAL"                  |"INSURANCEPROVIDER"  |"BILLINGAMOUNT"       |"ROOMNUMBER"  |"ADMISSIONTYPE"  |"DISCHARGEDATE"  |"MEDICATION"  |"TESTRESULTS"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Bobby JacksOn  |30     |Male      |B-           |Cancer              |2024-01-31         |Matthew Smith     |Sons and Miller             |Blue Cross           |18856.28130597815500  |328           |

In [6]:
# Standardize names to proper case
standardized_names_df = healthcare_data_df.with_column("standardized_name", F.initcap("NAME"))
standardized_names_df.select("NAME", "standardized_name").show(5)

---------------------------------------
|"NAME"         |"STANDARDIZED_NAME"  |
---------------------------------------
|Bobby JacksOn  |Bobby Jackson        |
|LesLie TErRy   |Leslie Terry         |
|DaNnY sMitH    |Danny Smith          |
|andrEw waTtS   |Andrew Watts         |
|adrIENNE bEll  |Adrienne Bell        |
---------------------------------------



In [23]:
# New DF with Standardize names
healthcare_data_df = healthcare_data_df.with_column("standardized_name", F.initcap("NAME"))

In [8]:
top_conditions = healthcare_data_df.group_by('MEDICALCONDITION').count().order_by('count', ascending=False).limit(5)
top_conditions.show()

--------------------------------
|"MEDICALCONDITION"  |"COUNT"  |
--------------------------------
|Arthritis           |9308     |
|Diabetes            |9304     |
|Hypertension        |9245     |
|Obesity             |9231     |
|Cancer              |9227     |
--------------------------------



In [9]:
# Unique counts for blood type, medical condition, and insurance provider
unique_counts_df = healthcare_data_df.select(
    F.count_distinct("BLOODTYPE").alias("unique_bloodtypes"),
    F.count_distinct("MEDICALCONDITION").alias("unique_medical_conditions"),
    F.count_distinct("INSURANCEPROVIDER").alias("unique_insurance_providers")
)
unique_counts_df.show()

------------------------------------------------------------------------------------
|"UNIQUE_BLOODTYPES"  |"UNIQUE_MEDICAL_CONDITIONS"  |"UNIQUE_INSURANCE_PROVIDERS"  |
------------------------------------------------------------------------------------
|8                    |6                            |5                             |
------------------------------------------------------------------------------------



In [10]:
# Create a new column 'age_group' with age bins of 10 years
healthcare_data_df = healthcare_data_df.with_column(
    "age_group",
    F.when((F.col("AGE") >= 0) & (F.col("AGE") <= 9), '0-9')
    .when((F.col("AGE") >= 10) & (F.col("AGE") <= 19), '10-19')
    .when((F.col("AGE") >= 20) & (F.col("AGE") <= 29), '20-29')
    .when((F.col("AGE") >= 30) & (F.col("AGE") <= 39), '30-39')
    .when((F.col("AGE") >= 40) & (F.col("AGE") <= 49), '40-49')
    .when((F.col("AGE") >= 50) & (F.col("AGE") <= 59), '50-59')
    .when((F.col("AGE") >= 60) & (F.col("AGE") <= 69), '60-69')
    .when((F.col("AGE") >= 70) & (F.col("AGE") <= 79), '70-79')
    .when((F.col("AGE") >= 80) & (F.col("AGE") <= 89), '80-89')
    .when((F.col("AGE") >= 90) & (F.col("AGE") <= 99), '90-99')
    .otherwise('100+')
)

# Group by age_group and count number of patients by group
age_distribution = healthcare_data_df.group_by("age_group").count().order_by("age_group")
age_distribution.show()


-------------------------
|"AGE_GROUP"  |"COUNT"  |
-------------------------
|10-19        |1693     |
|20-29        |8000     |
|30-39        |8179     |
|40-49        |8130     |
|50-59        |8350     |
|60-69        |8177     |
|70-79        |8072     |
|80-89        |4899     |
-------------------------



In [11]:
# Average billing by insurance provider and admission type
avg_billing_df = healthcare_data_df.group_by("INSURANCEPROVIDER", "ADMISSIONTYPE").agg(
    F.to_char(F.avg("BILLINGAMOUNT"), "$999,999,999.00").alias("avg_billing")
).sort(F.col("avg_billing").desc())
avg_billing_df.show()

------------------------------------------------------------
|"INSURANCEPROVIDER"  |"ADMISSIONTYPE"  |"AVG_BILLING"     |
------------------------------------------------------------
|Cigna                |Elective         |      $26,013.70  |
|Aetna                |Urgent           |      $25,895.93  |
|UnitedHealthcare     |Emergency        |      $25,740.17  |
|Medicare             |Elective         |      $25,640.45  |
|Blue Cross           |Urgent           |      $25,640.44  |
|Blue Cross           |Elective         |      $25,628.97  |
|Medicare             |Urgent           |      $25,611.55  |
|Medicare             |Emergency        |      $25,596.09  |
|Blue Cross           |Emergency        |      $25,568.70  |
|Aetna                |Elective         |      $25,431.61  |
------------------------------------------------------------



In [12]:
# Sum billing amount by insurance provider
total_billing_df = healthcare_data_df.group_by("INSURANCEPROVIDER").agg(
    F.to_char(F.count("BILLINGAMOUNT"), '$999,999,999.00').alias("total_billing")
).sort(F.col("total_billing").desc()).limit(5)
total_billing_df.show()

------------------------------------------
|"INSURANCEPROVIDER"  |"TOTAL_BILLING"   |
------------------------------------------
|Cigna                |      $11,249.00  |
|Medicare             |      $11,154.00  |
|UnitedHealthcare     |      $11,125.00  |
|Blue Cross           |      $11,059.00  |
|Aetna                |      $10,913.00  |
------------------------------------------



In [13]:
# Drilling down to one hospital (Sons and Miller)
sons_and_miller_df = healthcare_data_df.filter(F.col("HOSPITAL") == 'Sons and Miller')
sons_and_miller_df.show(5)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"NAME"               |"AGE"  |"GENDER"  |"BLOODTYPE"  |"MEDICALCONDITION"  |"DATEOFADMISSION"  |"DOCTOR"               |"HOSPITAL"       |"INSURANCEPROVIDER"  |"BILLINGAMOUNT"       |"ROOMNUMBER"  |"ADMISSIONTYPE"  |"DISCHARGEDATE"  |"MEDICATION"  |"TESTRESULTS"  |"STANDARDIZED_NAME"  |"AGE_GROUP"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|Bobby JacksOn        |30     |Male      |B-           |Cancer              |2024-01-31    

In [14]:
# Patient count by medical condition for Sons and Miller
condition_count_df = sons_and_miller_df.group_by("MEDICALCONDITION").agg(F.count("*").alias("patient_count")).sort(F.col("patient_count").desc())
condition_count_df.show()

----------------------------------------
|"MEDICALCONDITION"  |"PATIENT_COUNT"  |
----------------------------------------
|Arthritis           |3                |
|Cancer              |2                |
|Hypertension        |1                |
|Diabetes            |1                |
----------------------------------------



In [15]:
# Average billing by admission type for Sons and Miller
avg_billing_admission_df = sons_and_miller_df.group_by("ADMISSIONTYPE").agg(
    F.to_char(F.avg("BILLINGAMOUNT"), "$999,999,999.00").alias("average_billing")
).sort(F.col("average_billing").desc())
avg_billing_admission_df.show()

---------------------------------------
|"ADMISSIONTYPE"  |"AVERAGE_BILLING"  |
---------------------------------------
|Emergency        |      $22,500.99   |
|Elective         |      $19,028.56   |
|Urgent           |      $18,856.28   |
---------------------------------------



In [16]:
# Distribution of patient count by age (in bins of 10 years) for Sons and Miller
age_group_distribution_df = sons_and_miller_df.select(
    F.when(F.col("AGE").between(0, 18), "0-18")
    .when(F.col("AGE").between(19, 35), "19-35")
    .when(F.col("AGE").between(36, 50), "36-50")
    .when(F.col("AGE").between(51, 65), "51-65")
    .otherwise("65+")
    .alias("age_group")
).group_by("age_group").agg(F.count("*").alias("patient_count")).sort("age_group")
age_group_distribution_df.show()

---------------------------------
|"AGE_GROUP"  |"PATIENT_COUNT"  |
---------------------------------
|19-35        |4                |
|36-50        |1                |
|65+          |2                |
---------------------------------



In [17]:
# Top 5 doctors with highest patient count at Sons and Miller
top_doctors_df = sons_and_miller_df.group_by("DOCTOR").agg(F.count("*").alias("patient_count")).sort(F.col("patient_count").desc()).limit(5)
top_doctors_df.show()

-------------------------------------------
|"DOCTOR"               |"PATIENT_COUNT"  |
-------------------------------------------
|Christopher Henderson  |2                |
|Lisa Guzman            |1                |
|Matthew Smith          |1                |
|Peter Young            |1                |
|Alex Banks             |1                |
-------------------------------------------



In [18]:
# Average length of stay by medical condition at Sons and Miller
avg_length_stay_df = sons_and_miller_df.group_by("MEDICALCONDITION").agg(
    F.trunc(F.avg(F.col("DISCHARGEDATE") - F.col("DATEOFADMISSION"))).alias("avg_length_of_stay_in_days")
).sort(F.col("avg_length_of_stay_in_days").desc())
avg_length_stay_df.show()

-----------------------------------------------------
|"MEDICALCONDITION"  |"AVG_LENGTH_OF_STAY_IN_DAYS"  |
-----------------------------------------------------
|Diabetes            |20                            |
|Arthritis           |16                            |
|Hypertension        |15                            |
|Cancer              |5                             |
-----------------------------------------------------



In [19]:
# Average billing amount by medical condition at Sons and Miller
avg_billing_condition_df = sons_and_miller_df.group_by("MEDICALCONDITION").agg(
    F.to_char(F.avg("BILLINGAMOUNT"), '$999,999,999.00').alias("avg_billing_amount")
).sort(F.col("avg_billing_amount").desc())
avg_billing_condition_df.show()

---------------------------------------------
|"MEDICALCONDITION"  |"AVG_BILLING_AMOUNT"  |
---------------------------------------------
|Diabetes            |      $31,055.69      |
|Arthritis           |      $19,622.13      |
|Cancer              |      $18,205.62      |
|Hypertension        |      $17,111.59      |
---------------------------------------------



In [20]:
# Sum billing amount by insurance provider at Sons and Miller
total_billing_df = sons_and_miller_df.group_by("INSURANCEPROVIDER").agg(
    F.to_char(F.count("BILLINGAMOUNT"), '$999,999,999.00').alias("total_billing")
).sort(F.col("total_billing").desc()).limit(5)
total_billing_df.show()

------------------------------------------
|"INSURANCEPROVIDER"  |"TOTAL_BILLING"   |
------------------------------------------
|Medicare             |           $3.00  |
|Blue Cross           |           $2.00  |
|Aetna                |           $1.00  |
|UnitedHealthcare     |           $1.00  |
------------------------------------------



In [21]:
# Correlation between age and billing amount at Sons and Miller
age_billing_corr_df = sons_and_miller_df.select(F.corr("AGE", "BILLINGAMOUNT").alias("age_billing_correlation"))
age_billing_corr_df.show()

-----------------------------
|"AGE_BILLING_CORRELATION"  |
-----------------------------
|-0.19975700313866945       |
-----------------------------

